In [71]:
import datetime 
import json

from typing import Literal, Any
from time import sleep
from pathlib import Path

import requests

HOST = "https://gestiona7.madrid.org"
FILENAME_SUMMON = "nsus_rest_ares/v1/SustInterinos/citados"
FILENAME_ASSIGN = "nsus_rest_ares/v1/ActosPublicosSustituciones/resolucion"

URL_SUMMON = f"{HOST}/{FILENAME_SUMMON}"
URL_ASSIGN = f"{HOST}/{FILENAME_ASSIGN}"

SECTION = "0590"
SPECIALTIES = (
    "006",
    "007",
    "008",
    "019",
    "102",
    "103",
    "115",
    "116",
    "117",
)

start_date = datetime.date(2022, 9, 1)
today = datetime.date.today()

In [58]:
def scrap_ares_summon(
    list_type: Literal["O", "V", "E"],
    specialty: str,
    date: datetime.date,
) -> list[dict[str, Any]]:
    fcpublico = date.strftime("%Y-%m-%d")
    cfunc = f"{SECTION}{specialty}"
    itTipoActo = list_type
    body = {
        "fcpublico": fcpublico,
        "cfunc": cfunc,
        "itTipoActo": itTipoActo
    }
    response = requests.post(URL_SUMMON, json=body)
    if response.status_code != 200:
        print(f"Error {response.status_code} for {fcpublico} {cfunc} {itTipoActo}")
        return []
    return response.json()

def scrap_summon_specialty(
    specialty: str,
    list_type: Literal["O", "V", "E"],
    date_from: datetime.date,
    date_to: datetime.date,
    sleep_time: float = 0.0,
) -> list[dict[str, Any]]:
    total_days = (date_to - date_from).days + 1
    dates = [date_from + datetime.timedelta(days=i) for i in range(total_days)]
    
    data = []
    for date in dates:
        date_data = scrap_ares_summon(list_type, specialty, date)
        data.extend(date_data)
        sleep(sleep_time)
    
    return data

def scrap_summon_list(
    list_type: Literal["O", "V", "E"],
    date_from: datetime.date,
    date_to: datetime.date,
    sleep_time: float = 0.0,
) -> list[dict[str, Any]]:
    data = []
    for specialty in SPECIALTIES:
        specialty_data = scrap_summon_specialty(specialty, list_type, date_from, date_to, sleep_time)
        data.extend(specialty_data)
    return data

def scrap_summon_list_to_json(
    list_type: Literal["O", "V", "E"],
    date_from: datetime.date,
    date_to: datetime.date,
    filename: str,
    sleep_time: float = 0.0,
) -> None:
    data = scrap_summon_list(list_type, date_from, date_to, sleep_time)
    with Path(filename).open("w") as f:
        json.dump(data, f)

In [59]:
scrap_summon_list_to_json("O", start_date, today, f"data/raw/summon-O.json", 0.0)
scrap_summon_list_to_json("V", start_date, today, f"data/raw/summon-V.json", 0.0)
scrap_summon_list_to_json("E", start_date, today, f"data/raw/summon-E.json", 0.0)

In [85]:
def scrap_ares_assign(
    list_type: Literal["O", "V", "E"],
    specialty: str,
    date: datetime.date,
) -> list[dict[str, Any]]:
    fcpublico = date.strftime("%Y-%m-%d")
    cfunc = f"{SECTION}{specialty}"
    ittipoacto = list_type
    body = {
        "fcpublico": fcpublico,
        "cfunc": cfunc,
        "ittipoacto": ittipoacto
    }
    response = requests.post(URL_ASSIGN, json=body)
    if response.status_code != 200:
        print(f"Error {response.status_code} for {fcpublico} {cfunc} {ittipoacto}")
        return []
    return response.json()


def scrap_assign_specialty(
    specialty: str,
    list_type: Literal["O", "V", "E"],
    date_from: datetime.date,
    date_to: datetime.date,
    sleep_time: float = 0.0,
) -> list[dict[str, Any]]:
    print(f"Scraping {specialty} {list_type} {date_from} {date_to}")
    total_days = (date_to - date_from).days + 1
    dates = [date_from + datetime.timedelta(days=i) for i in range(total_days)]
    
    data = []
    for date in dates:
        date_data = scrap_ares_assign(list_type, specialty, date)
        data.extend(date_data)
        sleep(sleep_time)
    
    return data

def scrap_assign_list(
    list_type: Literal["O", "V", "E"],
    date_from: datetime.date,
    date_to: datetime.date,
    sleep_time: float = 0.0,
) -> list[dict[str, Any]]:
    data = []
    for specialty in SPECIALTIES:
        specialty_data = scrap_assign_specialty(specialty, list_type, date_from, date_to, sleep_time)
        data.extend(specialty_data)
    return data

def scrap_assign_list_to_json(
    list_type: Literal["O", "V", "E"],
    date_from: datetime.date,
    date_to: datetime.date,
    filename: str,
    sleep_time: float = 0.0,
) -> None:
    data = scrap_assign_list(list_type, date_from, date_to, sleep_time)
    with Path(filename).open("w") as f:
        json.dump(data, f)        

In [86]:
scrap_assign_list_to_json("O", start_date, today, f"data/raw/assign-O.json", 0.0)
scrap_assign_list_to_json("V", start_date, today, f"data/raw/assign-V.json", 0.0)
scrap_assign_list_to_json("E", start_date, today, f"data/raw/assign-E.json", 0.0)

wtf
Scraping 006 O 2022-09-01 2023-02-26
